In [37]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.models import load_model

In [2]:
places = pd.read_csv('https://raw.githubusercontent.com/anisimnida/Spin-Cycle/main/machine-learning/cleaned%20data/laundry.csv')
places

,ID Laundry,Name,Fulladdress,Categories,phone,Review Count,Average Rating,Review URL,Google Maps URL,Latitude,Longitude,Website,Opening Hours,Featured Image,Layanan
0,1,Laundryklin Sudirman ( Laundry Antar Jemput Ba...,"Jl. Jend. Sudirman No.486, Ciroyom, Kec. Andir...",Layanan Binatu,+62 851-7335-8871,151,4.8,https://search.google.com/local/reviews?placei...,https://www.google.com/maps?cid=94604019837086...,-6.918552,107.586449,NaN,"Sabtu: [07.00-21.00], Minggu: [07.00-21.00], S...",https://lh5.googleusercontent.com/p/AF1QipMUOc...,"just wash, dry cleaning, iron wash"
1,2,The Laundry Room,"Jl. Ciroyom No.95/77, Dungus Cariang, Kec. And...",Layanan Binatu,+62 22 20562320,3,5.0,https://search.google.com/local/reviews?placei...,https://www.google.com/maps?cid=11162829296077...,-6.909928,107.580838,https://www.instagram.com/the.laundry.room.ban...,"Sabtu: [Buka 24 jam], Minggu: [Buka 24 jam], S...",https://lh5.googleusercontent.com/p/AF1QipOt-O...,"just wash, dry cleaning, iron wash"
2,3,Nadhif Laundry ciroyom bandung,"Gg. Sastra No.272/78, Dungus Cariang, Kec. And...",Binatu,+62 22 6036296,6,4.7,https://search.google.com/local/reviews?placei...,https://www.google.com/maps?cid=16206958779682...,-6.915349,107.585807,NaN,"Sabtu: [08.00-20.00], Minggu: [08.00-20.00], S...",https://lh5.googleusercontent.com/p/AF1QipPN10...,"just wash, iron wash"
3,4,Laundry Club Rajawali,"Jl. Rajawali Barat No.25, Garuda, Kec. Andir, ...",Layanan Binatu,+62 851-7324-7944,33,4.9,https://search.google.com/local/reviews?placei...,https://www.google.com/maps?cid=10211050715543...,-6.911077,107.570868,https://bit.ly/customer_laundryclub_rajawali,"Sabtu: [07.00-20.00], Minggu: [09.00-17.00], S...",https://lh5.googleusercontent.com/p/AF1QipMERU...,"just wash, dry cleaning, iron wash"
4,5,EZpress Home Laundry,"Jl. Durman No.4, Kb. Jeruk, Kec. Andir, Kota B...",Layanan Binatu,+62 823-2020-9878,50,4.2,https://search.google.com/local/reviews?placei...,https://www.google.com/maps?cid=89067263696846...,-6.919917,107.599841,NaN,"Sabtu: [07.00-18.00], Minggu: [Tutup], Senin: ...",https://lh5.googleusercontent.com/p/AF1QipOy4x...,"just wash, dry cleaning, iron wash"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1221,1222,Vanili Laundry,"Jl. Vanili Raya No.99, Mangga, Kec. Medan Tunt...",Layanan Binatu,NaN,1,5.0,https://search.google.com/local/reviews?placei...,https://www.google.com/maps?cid=13474658465171...,3.508912,98.632010,0,NaN,https://streetviewpixels-pa.googleapis.com/v1/...,"just wash, dry cleaning, iron wash"
1222,1223,Laundry The Daily Wash dr.mansur,"Jl. Dr. Mansyur No.27, Padang Bulan Selayang I...",Binatu,NaN,16,4.3,https://search.google.com/local/reviews?placei...,https://www.google.com/maps?cid=91915042862606...,3.567332,98.651193,0,NaN,https://lh5.googleusercontent.com/p/AF1QipPZbY...,"just wash, iron wash"
1223,1224,Tetanggamu Laundry,"Jl. Prof. H. M. Yamin No.17b, Perintis, Kec. M...","Binatu, Binatu Cucian Kering",+62 812-6511-5252,71,5.0,https://search.google.com/local/reviews?placei...,https://www.google.com/maps?cid=17346466435953...,3.594387,98.680243,0,"Minggu: [08.30-19.00], Senin: [08.30-19.00], S...",https://lh5.googleusercontent.com/p/AF1QipN-8x...,"just wash, dry cleaning, iron wash"
1224,1225,Mutiara Laundry Express Powered By DEXTER,"Jl. Arief Rahman Hakim No.136, RT.02, Ps. Mera...",Binatu,NaN,9,4.6,https://search.google.com/local/reviews?placei...,https://www.google.com/maps?cid=33600404797552...,3.571973,98.703399,0,"Minggu: [09.00-22.00], Senin: [09.00-22.00], S...",https://lh5.googleusercontent.com/p/AF1QipMi-K...,"just wash, iron wash"


In [3]:
# Contoh data pengguna
user = {'Latitude': 3.504912, 'Longitude': 98.51700, 'Select Layanan': ['just wash']}

In [5]:
# Label encoding untuk kategori tempat
label_encoder = LabelEncoder()
places['category_encoded'] = label_encoder.fit_transform(places['Layanan'])

In [4]:
# Menghitung jarak antara pengguna dan tempat menggunakan Haversine
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0  # Radius Bumi dalam kilometer
    dlat = np.radians(lat2 - lat1)
    dlon = np.radians(lon2 - lon1)
    a = np.sin(dlat / 2) ** 2 + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.sin(dlon / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance = R * c
    return distance

places['distance'] = places.apply(lambda row: haversine(user['Latitude'], user['Longitude'], row['Latitude'], row['Longitude']), axis=1)


In [6]:
# Menggabungkan kategori preferensi pengguna
user_preferred_categories = ' '.join(user['Select Layanan'])

# Menyiapkan data untuk training
X = places[['distance', 'category_encoded']].values
y = places['category_encoded'].values

# Normalisasi data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Membagi data menjadi training dan test set
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [7]:
# Membangun model neural network
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='linear'))

# Mengompilasi model
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse'])

# Melatih model
model.fit(X_train, y_train, epochs=50, batch_size=8, validation_split=0.2)


Epoch 1/50
98/98 [==============================] - 1s 4ms/step - loss: 7.3604 - mse: 7.3604 - val_loss: 0.3413 - val_mse: 0.3413
Epoch 2/50
98/98 [==============================] - 0s 3ms/step - loss: 0.2050 - mse: 0.2050 - val_loss: 0.0774 - val_mse: 0.0774
Epoch 3/50
98/98 [==============================] - 0s 3ms/step - loss: 0.0886 - mse: 0.0886 - val_loss: 0.0353 - val_mse: 0.0353
Epoch 4/50
98/98 [==============================] - 0s 2ms/step - loss: 0.0605 - mse: 0.0605 - val_loss: 0.0203 - val_mse: 0.0203
Epoch 5/50
98/98 [==============================] - 0s 2ms/step - loss: 0.0501 - mse: 0.0501 - val_loss: 0.0172 - val_mse: 0.0172
Epoch 6/50
98/98 [==============================] - 0s 2ms/step - loss: 0.0417 - mse: 0.0417 - val_loss: 0.0146 - val_mse: 0.0146
Epoch 7/50
98/98 [==============================] - 0s 3ms/step - loss: 0.0358 - mse: 0.0358 - val_loss: 0.0124 - val_mse: 0.0124
Epoch 8/50
98/98 [==============================] - 0s 3ms/step - loss: 0.0299 - mse: 0.02

In [8]:
def recommendation_system(user_latitude, user_longitude, selected_service, places, model, scaler):
    # Menghitung jarak antara pengguna dan tempat menggunakan Haversine
    def haversine(lat1, lon1, lat2, lon2):
        R = 6371.0  # Radius Bumi dalam kilometer
        dlat = np.radians(lat2 - lat1)
        dlon = np.radians(lon2 - lon1)
        a = np.sin(dlat / 2) ** 2 + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.sin(dlon / 2) ** 2
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
        distance = R * c
        return distance

    # Filter tempat berdasarkan layanan yang dipilih
    selected_places = places[places['Layanan'] == selected_service]

    # Hitung jarak dari pengguna ke tempat-tempat yang dipilih
    selected_places['distance'] = selected_places.apply(lambda row: haversine(user_latitude, user_longitude, row['Latitude'], row['Longitude']), axis=1)

    # Menyiapkan data untuk prediksi
    X = selected_places[['distance', 'category_encoded']].values

    # Normalisasi data
    X_scaled = scaler.transform(X)

    # Melakukan prediksi
    predictions = model.predict(X_scaled)

    # Menggabungkan prediksi dengan data tempat
    predictions_df = pd.DataFrame(predictions, columns=['predicted_category'])
    results_df = pd.concat([selected_places, predictions_df], axis=1)

    # Mengurutkan berdasarkan kategori yang diprediksi dan jarak
    results_df = results_df.sort_values(by=['predicted_category', 'distance'])

    return results_df[['Name', 'Fulladdress', 'Layanan', 'distance']]


In [10]:
# Panggil fungsi rekomendasi
lat_user = -6.918552
long_user = 107.586449
selected_service = ['dry cleaning']  # Ubah dengan layanan yang dipilih
recommended_places = recommendation_system(lat_user, long_user, selected_service, places, model, scaler)
print(recommended_places)


ValueError: ('Lengths must match to compare', (1226,), (1,))

In [11]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense

In [12]:
places = pd.read_csv('https://raw.githubusercontent.com/anisimnida/Spin-Cycle/main/machine-learning/cleaned%20data/laundry.csv')
places

,ID Laundry,Name,Fulladdress,Categories,phone,Review Count,Average Rating,Review URL,Google Maps URL,Latitude,Longitude,Website,Opening Hours,Featured Image,Layanan
0,1,Laundryklin Sudirman ( Laundry Antar Jemput Ba...,"Jl. Jend. Sudirman No.486, Ciroyom, Kec. Andir...",Layanan Binatu,+62 851-7335-8871,151,4.8,https://search.google.com/local/reviews?placei...,https://www.google.com/maps?cid=94604019837086...,-6.918552,107.586449,NaN,"Sabtu: [07.00-21.00], Minggu: [07.00-21.00], S...",https://lh5.googleusercontent.com/p/AF1QipMUOc...,"just wash, dry cleaning, iron wash"
1,2,The Laundry Room,"Jl. Ciroyom No.95/77, Dungus Cariang, Kec. And...",Layanan Binatu,+62 22 20562320,3,5.0,https://search.google.com/local/reviews?placei...,https://www.google.com/maps?cid=11162829296077...,-6.909928,107.580838,https://www.instagram.com/the.laundry.room.ban...,"Sabtu: [Buka 24 jam], Minggu: [Buka 24 jam], S...",https://lh5.googleusercontent.com/p/AF1QipOt-O...,"just wash, dry cleaning, iron wash"
2,3,Nadhif Laundry ciroyom bandung,"Gg. Sastra No.272/78, Dungus Cariang, Kec. And...",Binatu,+62 22 6036296,6,4.7,https://search.google.com/local/reviews?placei...,https://www.google.com/maps?cid=16206958779682...,-6.915349,107.585807,NaN,"Sabtu: [08.00-20.00], Minggu: [08.00-20.00], S...",https://lh5.googleusercontent.com/p/AF1QipPN10...,"just wash, iron wash"
3,4,Laundry Club Rajawali,"Jl. Rajawali Barat No.25, Garuda, Kec. Andir, ...",Layanan Binatu,+62 851-7324-7944,33,4.9,https://search.google.com/local/reviews?placei...,https://www.google.com/maps?cid=10211050715543...,-6.911077,107.570868,https://bit.ly/customer_laundryclub_rajawali,"Sabtu: [07.00-20.00], Minggu: [09.00-17.00], S...",https://lh5.googleusercontent.com/p/AF1QipMERU...,"just wash, dry cleaning, iron wash"
4,5,EZpress Home Laundry,"Jl. Durman No.4, Kb. Jeruk, Kec. Andir, Kota B...",Layanan Binatu,+62 823-2020-9878,50,4.2,https://search.google.com/local/reviews?placei...,https://www.google.com/maps?cid=89067263696846...,-6.919917,107.599841,NaN,"Sabtu: [07.00-18.00], Minggu: [Tutup], Senin: ...",https://lh5.googleusercontent.com/p/AF1QipOy4x...,"just wash, dry cleaning, iron wash"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1221,1222,Vanili Laundry,"Jl. Vanili Raya No.99, Mangga, Kec. Medan Tunt...",Layanan Binatu,NaN,1,5.0,https://search.google.com/local/reviews?placei...,https://www.google.com/maps?cid=13474658465171...,3.508912,98.632010,0,NaN,https://streetviewpixels-pa.googleapis.com/v1/...,"just wash, dry cleaning, iron wash"
1222,1223,Laundry The Daily Wash dr.mansur,"Jl. Dr. Mansyur No.27, Padang Bulan Selayang I...",Binatu,NaN,16,4.3,https://search.google.com/local/reviews?placei...,https://www.google.com/maps?cid=91915042862606...,3.567332,98.651193,0,NaN,https://lh5.googleusercontent.com/p/AF1QipPZbY...,"just wash, iron wash"
1223,1224,Tetanggamu Laundry,"Jl. Prof. H. M. Yamin No.17b, Perintis, Kec. M...","Binatu, Binatu Cucian Kering",+62 812-6511-5252,71,5.0,https://search.google.com/local/reviews?placei...,https://www.google.com/maps?cid=17346466435953...,3.594387,98.680243,0,"Minggu: [08.30-19.00], Senin: [08.30-19.00], S...",https://lh5.googleusercontent.com/p/AF1QipN-8x...,"just wash, dry cleaning, iron wash"
1224,1225,Mutiara Laundry Express Powered By DEXTER,"Jl. Arief Rahman Hakim No.136, RT.02, Ps. Mera...",Binatu,NaN,9,4.6,https://search.google.com/local/reviews?placei...,https://www.google.com/maps?cid=33600404797552...,3.571973,98.703399,0,"Minggu: [09.00-22.00], Senin: [09.00-22.00], S...",https://lh5.googleusercontent.com/p/AF1QipMi-K...,"just wash, iron wash"


In [13]:
# Contoh data pengguna
user = {'Latitude': 3.504912, 'Longitude': 98.51700, 'Select Layanan': 'just wash'}

In [15]:
# Split layanan menjadi kolom dengan one-hot encoding
services = places['Layanan'].str.get_dummies(sep=', ')

In [17]:
# Gabungkan dengan DataFrame asli
places = pd.concat([places, services], axis=1)

In [18]:
# Filter data berdasarkan layanan yang dipilih
selected_service = user['Select Layanan']
filtered_places = places[places[selected_service] == 1]

In [19]:
# Menghitung jarak antara pengguna dan tempat menggunakan Haversine
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0  # Radius Bumi dalam kilometer
    dlat = np.radians(lat2 - lat1)
    dlon = np.radians(lon2 - lon1)
    a = np.sin(dlat / 2) ** 2 + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.sin(dlon / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance = R * c
    return distance

filtered_places['distance'] = filtered_places.apply(lambda row: haversine(user['Latitude'], user['Longitude'], row['Latitude'], row['Longitude']), axis=1)


<ipython-input-19-e9a5f57f65c4>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_places['distance'] = filtered_places.apply(lambda row: haversine(user['Latitude'], user['Longitude'], row['Latitude'], row['Longitude']), axis=1)


In [20]:
# Menyiapkan data untuk training
X = filtered_places[['distance']].values
y = filtered_places[selected_service].values

# Normalisasi data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Membagi data menjadi training dan test set
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [21]:
# Membangun model neural network
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Mengompilasi model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Melatih model
model.fit(X_train, y_train, epochs=50, batch_size=8, validation_split=0.2)


Epoch 1/50
98/98 [==============================] - 2s 6ms/step - loss: 0.3394 - accuracy: 1.0000 - val_loss: 0.0610 - val_accuracy: 1.0000
Epoch 2/50
98/98 [==============================] - 0s 3ms/step - loss: 0.0215 - accuracy: 1.0000 - val_loss: 0.0071 - val_accuracy: 1.0000
Epoch 3/50
98/98 [==============================] - 0s 3ms/step - loss: 0.0043 - accuracy: 1.0000 - val_loss: 0.0026 - val_accuracy: 1.0000
Epoch 4/50
98/98 [==============================] - 0s 3ms/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 0.0013 - val_accuracy: 1.0000
Epoch 5/50
98/98 [==============================] - 0s 2ms/step - loss: 0.0010 - accuracy: 1.0000 - val_loss: 8.1876e-04 - val_accuracy: 1.0000
Epoch 6/50
98/98 [==============================] - 0s 3ms/step - loss: 4.6458e-04 - accuracy: 1.0000 - val_loss: 2.0887e-04 - val_accuracy: 1.0000
Epoch 7/50
98/98 [==============================] - 0s 2ms/step - loss: 1.2384e-04 - accuracy: 1.0000 - val_loss: 7.6146e-05 - val_accuracy: 1.0000


In [40]:
model.save('model_service.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [34]:
def recommendation_system(user_latitude, user_longitude, selected_service, places, model, scaler):
    # Filter data berdasarkan layanan yang dipilih
    filtered_places = places[places[selected_service] == 1]

    # Hitung jarak dari pengguna ke tempat-tempat yang dipilih
    filtered_places['distance'] = filtered_places.apply(lambda row: haversine(user_latitude, user_longitude, row['Latitude'], row['Longitude']), axis=1)

    # Menyiapkan data untuk prediksi
    X = filtered_places[['distance']].values

    # Normalisasi data
    X_scaled = scaler.transform(X)

    # Melakukan prediksi
    predictions = model.predict(X_scaled)

    # Menambahkan kolom prediksi ke DataFrame
    filtered_places['predicted_prob'] = predictions

    # Mengurutkan tempat berdasarkan probabilitas prediksi
    filtered_places = filtered_places.sort_values(by='distance')
    filtered_places = filtered_places.head(30)

    return filtered_places[['Name', 'Fulladdress', 'phone', 'Average Rating', 'Google Maps URL', 'Website', 'Opening Hours', 'Featured Image', 'Layanan', 'distance']]

In [41]:
# Panggil fungsi rekomendasi
lat_user = 3.504912
long_user = 98.51700
selected_service = 'dry cleaning'  # Ubah dengan layanan yang dipilih

model = load_model('model_service.h5')
recommended_places = recommendation_system(lat_user, long_user, selected_service, places, model, scaler)
print(recommended_places)

24/24 [==============================] - 0s 1ms/step
                                               Name  \
1219                                   Baim Laundry   
1215                                 Darman Laundry   
1221                                 Vanili Laundry   
1070                                  Laundry Medan   
1198                               GS Laundry Medan   
1212                      KR5 Laundry and Dry clean   
1187                      Happy Laundry & Dry Clean   
1200                    Rainbow Laundry & Dry Clean   
1190                                  Medan Laundry   
1170                                  Faiqa Laundry   
1202                                Sembada laundry   
1199                    Noda Laundry & Wet Cleaning   
1188                                   Amal Laundry   
1201                                  David Laundry   
1128                          SYAKIRAH COIN LAUNDRY   
1090                                Parfait Laundry   
1197        

<ipython-input-34-47889f7f2f5c>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_places['distance'] = filtered_places.apply(lambda row: haversine(user_latitude, user_longitude, row['Latitude'], row['Longitude']), axis=1)
<ipython-input-34-47889f7f2f5c>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_places['predicted_prob'] = predictions
